In [ ]:
#--------------------------------
#    MODEL SIMULATIONS         
#--------------------------------

### Generate prior ensemble X
print('Generating the prior ensemble')
p1_in   = (np.array(p1)[xidx]).tolist() # mean of selected state vars
p2_in   = (np.array(p2)[xidx]).tolist() # std of -||-
pmin_in = (np.array(pmin)[xidx]).tolist() # lower bound of -||-
pmax_in = (np.array(pmax)[xidx]).tolist() # upper bound of -||-
# only perturb selected state vars; Xf dims will be [Ne,Nx_selected]
Xf_subset = create_ensemble.create_ensemble(n_ens,p1_in,p2_in,pmin_in,pmax_in,ens_gen)
print('     Shape of Xf with selected vars:   ',np.array(Xf_subset).shape)
# now construct the full prior ensemble in which selected vars are perturbed
# and masked variables are set to their true values for all ensemble members
Xf_mask = np.tile(x_true,n_ens).reshape(n_ens,len(x_true))
Xf_mask[:,xidx] = np.array(Xf_subset)
Xf = Xf_mask.tolist() # list in the form [[parms_mem{1}],...,[parms_mem{n_ens}]]
                        # where parms_mem{k} is made of the 11 state vars (CRM params)
print('     Shape of Xf with all vars:        ',np.array(Xf).shape)
print('     Prior mean (Xf.mean):             ',np.mean(Xf,axis=0)[xidx])
print('     Prior variance (Xf.var):          ',np.var(Xf,axis=0)[xidx])

### Mapping prior ensemble to observation space h(X) (i.e., run ensemble of CRM simulations)
print('Mapping prior ensemble to observation space (running the cloud model)')
runs = [] # full input list for the ensemble CRM runs
input_file_list = [input_file] * n_ens
output_file_list = [output_file] * n_ens
namelist_file_list = [namelist_file] * n_ens
run_num_list = list(range(1,n_ens+1))
runs = [list(x) for x in zip(input_file_list, output_file_list, namelist_file_list, run_num_list, Xf)]
print(f'    Full input to first ensemble member: {runs[0]}')
DASK_URL = 'scispark6.jpl.nasa.gov:8786'
parmode = 'par'
pmap = parmap.Parmap(master=DASK_URL, mode=parmode, numWorkers=num_Workers)
HXf = pmap(runcrm, runs)
print('    State vector for ens member Ne/2:  ', Xf[np.int32(n_ens/2)])
print('    ObSpace-mapped state vector for ens member Ne/2: ', HXf[np.int32(n_ens/2)])
print('    len(HXf), len(HXf[0]): ',len(HXf),len(HXf[0]))

In [ ]:
#--------------------------------
#    MODEL SIMULATIONS         
#--------------------------------

### Generate prior ensemble X
print('Generating the prior ensemble')
p1_in   = (np.array(p1)[xidx]).tolist() # mean of selected state vars
p2_in   = (np.array(p2)[xidx]).tolist() # std of -||-
pmin_in = (np.array(pmin)[xidx]).tolist() # lower bound of -||-
pmax_in = (np.array(pmax)[xidx]).tolist() # upper bound of -||-
# only perturb selected state vars; Xf dims will be [Ne,Nx_selected]
Xf_subset = create_ensemble.create_ensemble(n_ens, p1_in, p2_in, pmin_in, pmax_in, ens_gen)
print('     Shape of Xf with selected vars:   ', np.array(Xf_subset).shape)

# Now construct the full prior ensemble in which selected vars are perturbed
# and masked variables are set to their true values for all ensemble members
Xf_mask = np.tile(x_true, n_ens).reshape(n_ens, len(x_true))
Xf_mask[:, xidx] = np.array(Xf_subset)
Xf = Xf_mask.tolist()  # List in the form [[parms_mem{1}],...,[parms_mem{n_ens}]]
                       # Where parms_mem{k} is made of the 11 state vars (CRM params)

print('     Shape of Xf with all vars:        ', np.array(Xf).shape)
print('     Prior mean (Xf.mean):             ', np.mean(Xf, axis=0)[xidx])
print('     Prior variance (Xf.var):          ', np.var(Xf, axis=0)[xidx])

# ===================== NEW CODE TO SAVE INPUT PARAMETERS AS CSV FILE =====================
import pandas as pd

# Define the output CSV file path for input parameters
input_csv_path = './cloud_column_model/crm1d_input_params.csv'

# Define column names for the input parameters (these are the state variable names like 'as', 'bs', 'ag', etc.)
column_names_input = ['as', 'bs', 'ag', 'bg', 'N0r', 'N0s', 'N0g', 'rhos', 'rhog', 'qc0', 'qi0']

# Convert Xf (input ensemble) to a DataFrame for saving
Xf_df = pd.DataFrame(Xf, columns=column_names_input)

# Save the DataFrame to a CSV file
Xf_df.to_csv(input_csv_path, index=False)

print(f"Input parameters saved to {input_csv_path}")
# ======================================================================

### Mapping prior ensemble to observation space h(X) (i.e., run ensemble of CRM simulations)
# Mapping prior ensemble to observation space (running the cloud model)
print('Mapping prior ensemble to observation space (running the cloud model)')
runs = []  # Full input list for the ensemble CRM runs
input_file_list = [input_file] * n_ens
output_file_list = [output_file] * n_ens
namelist_file_list = [namelist_file] * n_ens
run_num_list = list(range(1, n_ens + 1))
runs = [list(x) for x in zip(input_file_list, output_file_list, namelist_file_list, run_num_list, Xf)]
print(f'    Full input to first ensemble member: {runs[0]}')

# Setup parallel processing
DASK_URL = 'scispark6.jpl.nasa.gov:8786'
parmode = 'par'
pmap = parmap.Parmap(master=DASK_URL, mode=parmode, numWorkers=num_Workers)

# Run the cloud model simulations for the ensemble members
HXf = pmap(runcrm, runs)

# Output state vectors for inspection
print('    State vector for ens member Ne/2:  ', Xf[np.int32(n_ens/2)])
print('    ObSpace-mapped state vector for ens member Ne/2: ', HXf[np.int32(n_ens/2)])
print('    len(HXf), len(HXf[0]): ', len(HXf), len(HXf[0]))

# ===================== NEW CODE TO SAVE HXf AS CSV FILE =====================

# Define the output CSV file path for the cloud model outputs
output_csv_path = './cloud_column_model/crm1d_output.csv'

# Define column names (adjust based on actual variables and time steps)
# Assuming 6 time steps and the following variables: pcp, acc, lwp, iwp, olr, osr
column_names = [f'{var}_t{t+1}' for t in range(6) for var in ['pcp', 'acc', 'lwp', 'iwp', 'olr', 'osr']]

# Convert HXf to a DataFrame
HXf_df = pd.DataFrame(HXf, columns=column_names)

# Save the DataFrame to a CSV file
HXf_df.to_csv(output_csv_path, index=False)

print(f"Output saved to {output_csv_path}")


In [ ]:
#------------------------
#    TRUE SIMULATION            
#------------------------
 
### Run the cloud model with the true parameters
print('*** TRUE SIMILATION ***')

# Path to the output file where true simulation results will be saved
true_output_file_path = './cloud_column_model/crm1d_true_output.csv'

# Run the cloud model with the true parameters
print('*** TRUE SIMULATION ***')
crm1d = cloud_column_model.CRM1DWrap(input_file, true_output_file_path, namelist_file, params=x_true, verbose=True)
y_true, crm_status = crm1d()  # Running the cloud model

# Save the true output to a CSV file (for future use)
column_names = [f'{var}_t{t+1}' for t in range(6) for var in ['pcp', 'acc', 'lwp', 'iwp', 'olr', 'osr']]
true_output_df = pd.DataFrame([y_true], columns=column_names)  # Assuming y_true is a flat list of length 36

true_output_df.to_csv(true_output_file_path, index=False)
print(f'True output saved to {true_output_file_path}')

# If needed, load the true output from the CSV
if os.path.exists(true_output_file_path):
    true_output_df = pd.read_csv(true_output_file_path)
    y_true = true_output_df.iloc[0].values  # Extract the first row (true output)

# Output the true simulation results
print(f'Output length: {len(y_true)}')
print('Output: ', y_true)
